In [55]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
import config
import os
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

### 读数据

In [52]:
train_filename = os.path.join(config.base_path, "data","train.xlsx")
test_filename = os.path.join(config.base_path, "data","测试A.xlsx")
df_train_raw = pd.read_excel(train_filename) #训练数据
df_test_raw = pd.read_excel(test_filename) #测试数据

df_train = df_train_raw.iloc[:,1:] 
df_train.index = df_train_raw.iloc[:,0].values
df_test = df_test_raw.iloc[:,1:]
df_test.index = df_test_raw.iloc[:,0].values

cols = df_train.columns
df_all = pd.concat([df_train, df_test])
df_all = df_all[cols]
print("number of train sample : %d" %df_train.shape[0])
print("number of test sample : %d" %df_test.shape[0])
print("number of features : %d" %df_test.shape[1])
print("number of all samples : %d" %df_all.shape[0])

number of train sample : 499
number of test sample : 100
number of features : 8027
number of all samples : 599


### 去除重复列和时间列

In [53]:
#去除掉全部为空的列
df_all.dropna(axis=1, how='all', inplace=True)
print("number of features : %d" %df_all.shape[1] )

cols = df_all.columns
cols_tool = list(filter(lambda x : "tool" in x.lower(), cols))
print(cols_tool)

# 去掉完全重复的列和时间列
dropped_col = []
for i in set(cols)-set(cols_tool):
    if len(set(df_all[i])) <= 1:
        dropped_col.append(i)
    if all([str(e).startswith("2017") or str(e).startswith("2016") for e in df_all[i][:20]]):
        dropped_col.append(i)
dropped_col.append("520X171")
df_all.loc[:,dropped_col].to_csv(os.path.join(config.base_path, "data", "重复的列和时间.csv"))
df_all = df_all.drop(labels=dropped_col, axis=1)
print("number of features : %d" %df_all.shape[1])

number of features : 7966
['TOOL_ID', 'Tool', 'TOOL_ID (#1)', 'TOOL_ID (#2)', 'TOOL_ID (#3)', 'Tool (#1)', 'Tool (#2)', 'tool', 'tool (#1)', 'TOOL', 'TOOL (#1)', 'Tool (#3)', 'TOOL (#2)']
number of features : 7168


### Label Encoder

In [57]:
le = LabelEncoder()
for col in cols_tool:
    tools = df_all[col].values
    df_all[col] = le.fit_transform(tools)
print(df_all[cols_tool[0]])

ID001    4
ID002    3
ID003    2
ID004    3
ID005    3
ID006    3
ID007    4
ID010    4
ID011    4
ID012    3
ID013    2
ID015    0
ID018    0
ID019    0
ID032    3
ID033    2
ID034    4
ID035    3
ID036    2
ID037    4
ID040    3
ID042    2
ID044    4
ID046    3
ID049    4
ID051    2
ID056    4
ID063    2
ID064    4
ID067    4
        ..
ID807    5
ID811    5
ID812    5
ID813    4
ID816    3
ID817    2
ID818    4
ID819    5
ID820    5
ID821    2
ID822    3
ID823    3
ID824    2
ID825    3
ID826    3
ID833    5
ID834    5
ID835    5
ID838    2
ID839    4
ID841    4
ID844    5
ID848    5
ID849    4
ID850    2
ID851    5
ID852    5
ID853    4
ID854    1
ID855    2
Name: TOOL_ID, Length: 599, dtype: int64
ID001    1
ID002    1
ID003    1
ID004    1
ID005    1
ID006    1
ID007    1
ID010    1
ID011    1
ID012    1
ID013    1
ID015    1
ID018    1
ID019    1
ID032    1
ID033    1
ID034    1
ID035    1
ID036    1
ID037    1
ID040    1
ID042    1
ID044    1
ID046    1
ID049    1
ID051    1
ID

### 缺失值填充和数据标准化

In [59]:
# 按照不同的TOOL类别填充均值
cols = df_all.columns
# cols_tool = list(filter(lambda x : "tool" in x.lower(), cols))
# print(cols_tool)

scaler = StandardScaler()
for i in range(len(cols_tool) - 1):
    df_tmp = df_all.loc[:,cols_tool[i]:cols_tool[i+1]].iloc[:,:-1]
    tools = df_tmp[cols_tool[i]]
    tool_set = list(set(tools))
    for k, tool in enumerate(tool_set):
        df_tool = df_tmp.loc[df_tmp[cols_tool[i]]==tool]
        mean_tool = df_tool.mean().fillna(0)
        df_tool = df_tool.fillna(mean_tool)
        
#         df_tool = pd.DataFrame(scaler.fit_transform(df_tool.values), index = df_tool.index, columns = df_tool.columns)

        print("--%s: %s done--" %(cols_tool[i], tool))
        if k == 0:
            tmp = df_tool
        else:
            tmp = pd.concat([tmp, df_tool])
    if i == 0:
        tool_frame = tmp
    else:
        tool_frame = pd.concat([tool_frame, tmp], axis = 1)
        
df_tmp = df_all.loc[:,cols_tool[-1]:]
tools = df_tmp[cols_tool[-1]]
tool_set = list(set(tools))
for k, tool in enumerate(tool_set):
    df_tool = df_tmp.loc[df_tmp[cols_tool[-1]]==tool]
    mean_tool = df_tool.mean().fillna(0)
    df_tool = df_tool.fillna(mean_tool)
    print("--%s: %s done--" %(cols_tool[-1], tool))
    if k == 0:
        tmp = df_tool
    else:
        tmp = pd.concat([tmp, df_tool])

tool_frame = pd.concat([tool_frame, tmp], axis = 1)

        
df_all_fillna = tool_frame[cols]

--TOOL_ID: 0 done--
--TOOL_ID: 1 done--
--TOOL_ID: 2 done--
--TOOL_ID: 3 done--
--TOOL_ID: 4 done--
--TOOL_ID: 5 done--
--Tool: 0 done--
--Tool: 1 done--
--TOOL_ID (#1): 0 done--
--TOOL_ID (#1): 1 done--
--TOOL_ID (#2): 0 done--
--TOOL_ID (#2): 1 done--
--TOOL_ID (#2): 2 done--
--TOOL_ID (#3): 0 done--
--TOOL_ID (#3): 1 done--
--Tool (#1): 0 done--
--Tool (#1): 1 done--
--Tool (#1): 2 done--
--Tool (#1): 3 done--
--Tool (#1): 4 done--
--Tool (#1): 5 done--
--Tool (#1): 6 done--
--Tool (#1): 7 done--
--Tool (#1): 8 done--
--Tool (#2): 0 done--
--Tool (#2): 1 done--
--Tool (#2): 2 done--
--tool: 0 done--
--tool: 1 done--
--tool: 2 done--
--tool: 3 done--
--tool (#1): 0 done--
--tool (#1): 1 done--
--tool (#1): 2 done--
--tool (#1): 3 done--
--tool (#1): 4 done--
--tool (#1): 5 done--
--tool (#1): 6 done--
--tool (#1): 7 done--
--tool (#1): 8 done--
--TOOL: 0 done--
--TOOL: 1 done--
--TOOL: 2 done--
--TOOL (#1): 0 done--
--TOOL (#1): 1 done--
--Tool (#3): 0 done--
--Tool (#3): 1 done--
--

In [83]:
#print(df_all_fillna.loc["ID563","312X211"])

df_train = df_all.loc[df_train.index,:]
df_test = df_all.loc[df_test.index,:]
print(df_train.columns)
print(df_test.index)
print("number of train sample : %d" %df_train.shape[0])
print("number of test sample : %d" %df_test.shape[0])
print("number of features : %d" %df_test.shape[1])

Index(['TOOL_ID', '210X1', '210X2', '210X3', '210X4', '210X5', '210X6',
       '210X7', '210X8', '210X9',
       ...
       '750X1444', '750X1445', '750X1446', '750X1447', '750X1448', '750X1449',
       '750X1450', '750X1451', '750X1452', 'Y'],
      dtype='object', length=7168)
Index(['ID716', 'ID717', 'ID719', 'ID720', 'ID721', 'ID722', 'ID723', 'ID724',
       'ID725', 'ID727', 'ID728', 'ID729', 'ID730', 'ID731', 'ID732', 'ID733',
       'ID734', 'ID735', 'ID736', 'ID737', 'ID738', 'ID739', 'ID740', 'ID741',
       'ID742', 'ID743', 'ID744', 'ID745', 'ID746', 'ID747', 'ID749', 'ID751',
       'ID752', 'ID753', 'ID757', 'ID759', 'ID760', 'ID761', 'ID762', 'ID763',
       'ID764', 'ID765', 'ID766', 'ID768', 'ID769', 'ID770', 'ID771', 'ID772',
       'ID773', 'ID774', 'ID776', 'ID777', 'ID778', 'ID779', 'ID781', 'ID783',
       'ID785', 'ID786', 'ID787', 'ID788', 'ID790', 'ID792', 'ID793', 'ID797',
       'ID798', 'ID801', 'ID802', 'ID803', 'ID804', 'ID806', 'ID807', 'ID811',
       'I

### 按照相关系数筛选

In [81]:
corr_values = []
k = 3000
for col in df_test.columns[:-1]:
    corr_values.append(abs(pearsonr(df_train[col].values,df_train['Y'])[0]))
corr_df = pd.DataFrame({'col':df_test.columns[:-1], 'corr_value':corr_values})
corr_df = corr_df.sort_values(by='corr_value',ascending=False)
selected = corr_df['col'].values[:k]

df_train_corr = df_train.loc[:, list(selected) + ['Y']]
df_test_corr = df_test.loc[:, list(selected)]

D:\Anaconda\Anaconda\lib\site-packages\scipy\stats\stats.py:5084: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0
D:\Anaconda\Anaconda\lib\site-packages\scipy\stats\stats.py:3029: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


### 写入文件

In [84]:
after_file = os.path.join(config.base_path, "data", "feature_selected_A_3000_category.xlsx")
with pd.ExcelWriter(after_file) as writer:
    df_train_corr.to_excel(writer,sheet_name = "train_data")
    df_test_corr.to_excel(writer, sheet_name = "test_data")